In [1]:
from app import models

In [2]:
data = models.Opinion.objects.filter()

In [3]:
len(data)

5302

In [4]:
stocks = models.Stocks.objects.filter()

In [5]:
stocks[0].stock

'زين السعودية'

In [6]:
data[11].text

'RT @L37612469: كيان يا ريح النفل وأطيب العود\n             يا اللي بلابل نجد باسمك تغني  !!!\n♥♥♥\n#إهداء\n#شكرا_لشاعرها http://t.co/zt1Daob86D'

In [7]:
stock_invalid = dict([(s.stock, []) for s in stocks])
for st in stocks:
    tws = models.Opinion.objects.filter(stock=st)
    print('Name:\t\t', st.stock)
    print('Total:\t\t', len(tws))
    c = 0
    for tw in tws:
        if st.stock not in tw.text:
            stock_invalid[st.stock].append(tw)
            c+=1
    print('Missing name:\t', len(stock_invalid[st.stock]))

Name:		 زين السعودية
Total:		 798
Missing name:	 588
Name:		 سابك
Total:		 1656
Missing name:	 18
Name:		 كيان
Total:		 2269
Missing name:	 73
Name:		 معدنية
Total:		 579
Missing name:	 186


In [2]:
print(len(datasetBuilder.trainSet))

2843


In [2]:
datasetBuilder.trainSet[:10]

[{'label': 'Negative',
  'stock': 'ﺱﺎﺒﻛ',
  'text': 'RT @RulaTarawneh: بعد الزيادات المقررة لاسعار مدخلات الانتاج ،ارتفاع تكلفة الانتاج ل #سابك 5% #كيان \xa08.5% #سافكو 8% #ينساب 6.5% . #السعودية',
  'words': ['الانتاج', 'تكلفة', 'ارتفاع']},
 {'label': 'Negative',
  'stock': 'ﻂﻴﺑﺓ ﻝﻼﺴﺘﺜﻣﺍﺭ',
  'text': 'سهم طيبه طال عمرك المضارب حسود',
  'words': ['حسود']},
 {'label': 'Negative',
  'stock': 'ﺎﻠﻨﻘﻟ ﺎﻠﺠﻣﺎﻌﻳ',
  'text': '#تداول النقل الجماعي يجنب نخصص زكوي في الربع الرابع بمقدار 29.2 مليون ريال لصالح مصلحة الزكاة.',
  'words': ['نخصص', 'يجنب']},
 {'label': 'Negative',
  'stock': 'ﺱﺎﺑ',
  'text': '#تخيل_مستقبل_المملكة بنك ساب يفلس بعد ١٠ سنوات ?',
  'words': ['يفلس']},
 {'label': 'Negative',
  'stock': 'ﺎﻠﻨﻘﻟ ﺎﻠﺠﻣﺎﻌﻳ',
  'text': '@tasistockk والله سهم الجماعي بدون تجاوز 18.10 السهم ليس للشراء',
  'words': ['تجاوز', 'بدون', 'للشراء', 'ليس']},
 {'label': 'Negative',
  'stock': 'ﺎﻠﻏﺍﺯ ﻭﺎﻠﺘﺼﻨﻴﻋ',
  'text': 'أكثر الشركات انخفاضا : الأبحاث و التسويق \xa06% مبرد ، صناعة الورق ، صادرات 3.5% ثمار 3%

In [1]:
from DatasetBuilder.DatasetBuilder import DatasetBuilder
import os
configFileDatasetBuilder = os.path.join('DatasetBuilder','Configurations','Configurations.xml')
                   
# The serialization file to save the dataset
datasetSerializationFile = os.path.join('DatasetBuilder','Output', 'dataset.bin')

# The XLSX file name for train set
xlsxTrainFileName = os.path.join('DatasetBuilder','Input','sentiment')


# Initialize the DatasetBuilder from serialization file
datasetBuilder = DatasetBuilder(configFileDatasetBuilder, [], datasetSerializationFile)
#datasetBuilder.trainSet = datasetBuilder.GetDatasetFromBackend(stock)
datasetBuilder.trainSet = datasetBuilder.GetSentimentDatasetFromXLSXFile(xlsxTrainFileName)

In [2]:
words_dict = {'Negative':[], 'Positive': [], 'Neutral': []}
for item in datasetBuilder.trainSet:
    words_dict[item['label']] += item['words']

In [3]:
for k in words_dict:
    words_dict[k] = list(set(words_dict[k]))

In [4]:
from FeaturesExtractor.FeaturesExtractor import FeaturesExtractor
from LanguageModel.LanguageModel import LanguageModel   

stock_model = {}
configFileLanguageModel_lexicon = os.path.join('LanguageModel', 'Configurations', 'Configurations_sentiment.xml')
stopWordsFileName = os.path.join('LanguageModel', 'Input', 'stop_words.txt')
linksDBFile = os.path.join('LanguageModel', 'Output', 'links_database.txt')
# The serialization file to save the model
languageModelSerializationFile = os.path.join('LanguageModel', 'Output', 'language_model.bin')

# Start the LanguageModel:

# Initialize the LanguageModel_Lexicon
stock_model['languageModel_lexicon'] = LanguageModel(configFileLanguageModel_lexicon, stopWordsFileName, languageModelSerializationFile, linksDBFile, datasetBuilder.trainSet)
stock_model['languageModel_lexicon'].BuildLanguageModel()
configFileFeaturesExtractor_Lexicon = os.path.join('FeaturesExtractor', 'Configurations', 'Configurations_sentiment.xml')
trainFeaturesSerializationFile = os.path.join('FeaturesExtractor', 'Output', 'train_features.bin')
trainLabelsSerializationFile = os.path.join('FeaturesExtractor', 'Output', 'train_labels.bin')

# Start the FeaturesExtractor:
#-----------------------------    
# Initialize the FeaturesExtractor _ Lexicon
trainFeaturesExtractor_Lexicon = FeaturesExtractor(configFileFeaturesExtractor_Lexicon, trainFeaturesSerializationFile, trainLabelsSerializationFile, stock_model['languageModel_lexicon'], datasetBuilder.trainSet, sentiment_features=True)
trainFeaturesExtractor_Lexicon.ExtractNumTfFeatures(words_dict)

Positive  2
Negative  1
Neutral  3


In [5]:
[i for i in trainFeaturesExtractor_Lexicon.features[0].items() if i[1] > 0 ]

[(10, 0.003177831885381874),
 (55, 0.004473367364774005),
 (57, 0.004505380386763192),
 (76, 0.005019712208397845),
 (568, 0.006726174474643525),
 (620, 0.006934731384335368),
 (711, 0.007053720372855124),
 (753, 0.007185256597147041),
 (1107, 0.007691455233986198),
 (1130, 0.007691455233986198),
 (1728, 0.008197653870825355),
 (2322, 0.00855680751932945),
 (2914, 0.01985671688302372),
 (3740, 0.009063006156168608),
 (4700, 0.00992835844151186),
 (5453, 0.00992835844151186),
 (6713, 0.00992835844151186),
 (9053, 0.00992835844151186),
 (9067, 0.00992835844151186),
 (9177, 0.00992835844151186),
 (9642, 0.00992835844151186),
 (11707, 4),
 (11709, 9),
 (11710, 15),
 (11711, 8)]

In [13]:
len(trainFeaturesExtractor_Lexicon.features[0])
trainFeaturesExtractor_Lexicon.labels[:4]

[1, 1, 1, 1]

In [8]:
print(set([len(i) for i in trainFeaturesExtractor_Lexicon.features]))
maxid =  max([max(i.keys()) for i in trainFeaturesExtractor_Lexicon.features])

{11710}


In [9]:
import random
from sklearn.preprocessing import OneHotEncoder
X= []
Y = []

for i, item in enumerate(trainFeaturesExtractor_Lexicon.features):    
    itemx = [0 for _ in range(maxid)]
    l = [0,0,0]
    l[trainFeaturesExtractor_Lexicon.labels[i]-1] = 1
    for j in trainFeaturesExtractor_Lexicon.features[i]:
        v = trainFeaturesExtractor_Lexicon.features[i][j]
        itemx[j-1] = v
    X.append(itemx)
    Y.append(trainFeaturesExtractor_Lexicon.labels[i])


In [10]:
print(len(X))
print(X[0])
print(Y[0])


2843
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.003177831885381874, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.004473367364774005, 0, 0, 0.004505380386763192, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.005019712208397845, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
import pickle
f = open('data/sent_dataset2.pkl', 'wb')
pickle.dump({'X': X, 'Y':Y}, f)